## EXPERIMENT TRACKING

#### ANWESHA PAUL <br> MDS202213

In [13]:
%run prepare.ipynb
%run train.ipynb

import pickle
import warnings
warnings.filterwarnings('ignore')

In [14]:
# Loading data
raw_data = load_data('emails.csv')

Sample of 5 rows from the DataFrame:
                                                   text  spam
530   Subject: out of office autoreply : just to her...     1
713   Subject: innovative and effective design for y...     1
1130  Subject: any med for your girl to be happy !  ...     1
3664  Subject: re : d - g energy  karla ,  we may us...     0
2290  Subject: re : stanford project  nick ,  thanks...     0

Shape of the DataFrame: (5728, 2)


In [15]:
# # Set up Git and DVC to track data
# git_dvc_setup()

In [17]:
# Splitting the data
X_train, X_val, X_test, y_train, y_val, y_test = split_data(raw_data, 101) #random state = 101
save_data(X_train, X_val, X_test, y_train, y_val, y_test)

In [18]:
# track_data("Tracking Data for version 1")

In [8]:
# Checkout first version
checkout_version(1)
load_and_check_data('Data/Training Data.csv', 'Data/Validation Data.csv', 'Data/Test Data.csv')

Updated 0 paths from b086e43
Building workspace index                              |5.00 [00:00,  180entry/s]
Comparing indexes                                     |5.00 [00:00,  439entry/s]
Applying changes                                      |0.00 [00:00,     ?file/s]

Train set:
no. of non-spam: 3148 
no. of spam: 989

Validation set:
no. of non-spam: 562 
no. of spam: 169

Test set:
no. of non-spam: 650 
no. of spam: 210


In [19]:
# Splitting the data again
X_train, X_val, X_test, y_train, y_val, y_test = split_data(raw_data, 404) #random state = 404
save_data(X_train, X_val, X_test, y_train, y_val, y_test)

In [20]:
# track_data("Tracking Data for version 2")

In [10]:
# Checkout second version
checkout_version(2)
load_and_check_data('Data/Training Data.csv', 'Data/Validation Data.csv', 'Data/Test Data.csv')

Updated 0 paths from fb1d46f


Building workspace index                              |5.00 [00:00,  138entry/s]
Comparing indexes                                     |5.00 [00:00,  436entry/s]
Applying changes                                      |0.00 [00:00,     ?file/s]

Train set:
no. of non-spam: 3154 
no. of spam: 983

Validation set:
no. of non-spam: 555 
no. of spam: 176

Test set:
no. of non-spam: 651 
no. of spam: 209


In [5]:
# Load data
train = pd.read_csv("Data/Training Data.csv")
val = pd.read_csv("Data/Validation Data.csv")
test = pd.read_csv("Data/Test Data.csv")

In [6]:
# Set up logging configuration
logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)

In [36]:
# # Convert text to vectors and store them
# convert_text_to_vectors(train, 'train_emb.pkl')
# convert_text_to_vectors(val, 'val_emb.pkl')
# convert_text_to_vectors(test, 'test_emb.pkl')

In [3]:
# Load vectors from files
with open('train_emb.pkl', 'rb') as file:
    train_emb = pickle.load(file)

with open('val_emb.pkl', 'rb') as file:
    val_emb = pickle.load(file)

with open('test_emb.pkl', 'rb') as file:
    test_emb = pickle.load(file)

### Train Models

#### 1. Logistic Regression

In [32]:
model_name = "Logistic_Regression"
clf = LogisticRegression()

param_grid_lr = {
    'C': [0.1, 1, 10],
    'solver': ['liblinear'], 
    'penalty': ['l1', 'l2'],
    'max_iter': [100, 200, 500]
}

lr_model, lr_acc, lr_aucpr = train_model(clf, param_grid_lr, train_emb, val_emb, test_emb, train['spam'], val['spam'], test['spam'])
log_model_metrics(lr_model, lr_acc, lr_aucpr, model_name)

version = get_model_version(model_name + "_Model")
print(f"Model Version: {version}")

2024/02/19 17:01:41 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '8c72c25c5754414098ef031ca9d5037d', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2024/02/19 17:01:41 INFO mlflow.sklearn.utils: Logging the 5 best runs, 5 runs for each unique parameter combination, and the 5 best runs for each unique parameter combination. To change this behavior, set the 'log_best_only' and 'log_all' parameters in the logging configuration.
Accuracy: 0.9790697674418605
AUCPR: 0.9636994332616768
Model Version: 1
Successfully registered model 'Logistic_Regression_Model'.
Created version '1' of model 'Logistic_Regression_Model'.


#### 2. Support Vector Classifier

In [31]:
model_name = "Support_Vector_Machine"
clf = SVC()

param_grid_svc = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'gamma': ['scale', 'auto']
}

svm_model, svm_acc, svm_aucpr = train_model(clf, param_grid_svc, train_emb, val_emb, test_emb, train['spam'], val['spam'], test['spam'])
log_model_metrics(svm_model, svm_acc, svm_aucpr, model_name)

version = get_model_version(model_name + "_Model")
print(f"Model Version: {version}")

2024/02/19 17:03:16 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'eb05c91928414e27a6c770d6ce144856', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2024/02/19 17:03:32 INFO mlflow.sklearn.utils: Logging the 5 best runs, 19 runs will be omitted.
Accuracy: 0.9872093023255814
AUCPR: 0.9795168674897935
Model Version: 1
Successfully registered model 'Support_Vector_Machine_Model'.
Created version '1' of model 'Support_Vector_Machine_Model'.


#### 3. Random Forest

In [10]:
model_name = "Random_Forest"
clf = RandomForestClassifier(n_estimators=300, max_depth=6, random_state=101)

param_grid_rf = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 6, 8, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

rf_model, rf_acc, rf_aucpr = train_model(clf, param_grid_rf, train_emb, val_emb, test_emb, train['spam'], val['spam'], test['spam'])
log_model_metrics(rf_model, rf_acc, rf_aucpr, model_name)

version = get_model_version(model_name + "_Model")
print(f"Model Version: {version}")

2024/02/19 17:24:14 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '87770c7756544a42992c3a7d9d305461', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2024/02/19 17:24:01 INFO mlflow.sklearn.utils: Logging the 5 best runs, 13 runs will be omitted.
Accuracy: 0.9630232558139535
AUCPR: 0.9493602134299807
Model Version: 1
Successfully registered model 'Random_Forest_Model'.
Created version '1' of model 'Random_Forest_Model'.


**CONCLUSION**: The SVM model demonstrated superior performance with the highest accuracy (98.72%) and AUCPR (0.9795) among the three models, indicating its effectiveness in this classification task.

### Ensemble Modeling: Stacking Classifier

In [11]:
model_name = "Stacking_Classifier"

estimators = [('lr', LogisticRegression()),
              ('svm', SVC()),
              ('rf', RandomForestClassifier(n_estimators=100, random_state=101))]

clf = StackingClassifier(estimators=estimators, final_estimator = LogisticRegression())

param_grid_stack = {
    'stack_method': ['auto', 'predict_proba'],
    'final_estimator': [LogisticRegression(), RandomForestClassifier()],
    'final_estimator__max_iter': [100, 200, 500],
    'cv': [2, 3, 5],
    'passthrough': [False, True]
}

stacking_model, stacking_acc, stacking_aucpr = train_model(clf, param_grid_stack, train_emb, val_emb, test_emb, train['spam'], val['spam'], test['spam'])
log_model_metrics(stacking_model, stacking_acc, stacking_aucpr, model_name)

version = get_model_version(model_name + "_Model")
print(f"Model Version: {version}")

2024/02/19 17:30:34 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '87770c7756544a42992c3a7d9d305461', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2024/02/19 17:36:41 INFO mlflow.sklearn.utils: Logging the 5 best runs, 13 runs will be omitted.
Accuracy: 0.9892093023255814
AUCPR: 0.981328739186211
Model Version: 1
Successfully registered model 'Stacking_Classifier_Model'.
Created version '1' of model 'Stacking_Classifier_Model'.
